In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bronze

In [0]:
%sql
CREATE or REPLACE TABLE bronze.sensors (
    sensor_id STRING,
    value DOUBLE,
    pollutant STRING,
    unit STRING,
    datetimeFrom STRING,
    datetimeTo STRING
)

In [0]:
%sql
INSERT INTO bronze.sensors

SELECT
    regexp_extract(input_file_name(), 'sensor_(\\d+)\\.json', 1) AS sensor_id,
    r.value AS value,
    r.parameter.name as pollutant,
    r.parameter.units as unit,
    r.period.datetimeFrom.local as datetimeFrom,
    r.period.datetimeTo.local as datetimeTo
FROM
    json.`abfss://airdata@airstorage.dfs.core.windows.net/2025/August/12th/Sensorinfo/*.json`
LATERAL VIEW EXPLODE(results) AS r

In [0]:
%sql
CREATE or REPLACE TABLE bronze.month (
    sensor_id STRING,
    value DOUBLE,
    pollutant STRING,
    unit STRING,
    datetimeFrom STRING,
    datetimeTo STRING
)

In [0]:
%sql
INSERT INTO bronze.month

SELECT
    regexp_extract(input_file_name(), 'sensor_(\\d+)\\.json', 1) AS sensor_id,
    r.value AS value,
    r.parameter.name as pollutant,
    r.parameter.units as unit,
    r.period.datetimeFrom.local as datetimeFrom,
    r.period.datetimeTo.local as datetimeTo
FROM
    json.`abfss://airdata@airstorage.dfs.core.windows.net/2025/August/15th/Sensorinfo/*.json`
LATERAL VIEW EXPLODE(results) AS r
EXCEPT
SELECT sensor_id, value, pollutant, unit, datetimeFrom, datetimeTo
FROM bronze.month